In [ ]:
!pip install yfinance pandas-datareader finance-datareader

In [ ]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

import numpy as np
import pandas as pd

import FinanceDataReader as fdr

In [ ]:
def get_return(code,k):
    df = fdr.DataReader(code,'2018')

    df['buy_at'] = (df['High'].shift(1) - df['Low'].shift(1))*k+df['Open']

    df['is_buy'] = np.where(df['High']>df['buy_at'],'buy','')

    df['sell_at'] = df['Open'].shift(-1)

    df = df[df['is_buy'] == 'buy']

    df['return'] = df['sell_at'] / df['buy_at']

    return df[['return']].cumprod().iloc[-1,-1] -1



In [ ]:
get_return('005930',0.43)

In [ ]:
df = pd.DataFrame()

for k in np.arange(0.4, 0.6, 0.01):
    doc = {
        'k':k,
        'return' : get_return('005930', k)
    }
    df = df.append(doc, ignore_index = True)

df.sort_values(by = 'return', ascending=False)

In [ ]:
def get_return_mf(code):
    df = fdr.DataReader(code, '2018')

    df = df.reset_index()
    df = df[['Date', 'Open']]

    df['Day'] = pd.to_datetime(df['Date']).dt.day_name()

    cond = (df['Day'] == 'Friday') | (df['Day'] == 'Monday')

    df = df[cond]

    if df.iloc[0, 2] == 'Friday':
        df = df.drop(index=df.index[0])

    if df.iloc[-1, 2] == 'Monday':
        df = df.drop(index=df.index[-1])

    cond = (df['Day'] == 'Monday') & (df['Day'].shift(-1) == 'Monday')
    df = df.drop(index=df[cond].index)

    cond = (df['Day'] == 'Friday') & (df['Day'].shift(1) == 'Friday')
    df = df.drop(index=df[cond].index)

    df['Open_fri'] = df['Open'].shift(-1)

    df = df[df['Day'] == 'Monday']

    df = df[['Open', 'Open_fri']]

    df.columns = ['buy_at', 'sell_at']

    df['return'] = df['sell_at'] / df['buy_at']

    cond = (df['sell_at'] != 0) & (df['buy_at'] != 0)
    df[cond]

    return df[['return']].cumprod().iloc[-1, -1] -1

In [ ]:
!pip install dart-fss

In [ ]:
import dart_fss as dart_fss
import pandas as pd

api_key = '852a7d11eaff31427834cceb019daf257a923c33'
dart_fss.set_api_key(api_key=api_key)

all = dart_fss.api.filings.get_corp_code()

df = pd.DataFrame(all)

df_listed = df[df['stock_code'].notnull()]



In [ ]:
df = pd.DataFrame()
for row in df_listed.sample(10)[['stock_code','corp_name']].itertuples():
  try:
    doc = {
      'name' : row[2],
      'return' : get_return_mf(row[1])
    }
    df = df.append(doc, ignore_index = True)
  except:
    print(f'error - {row[2]}')  

df.sort_values(by= 'return', ascending = False)    


"001490" invalid symbol or has no data
error - 천지산업


<ipython-input-71-07efbceb0bb1>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc, ignore_index = True)
<ipython-input-71-07efbceb0bb1>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc, ignore_index = True)
<ipython-input-71-07efbceb0bb1>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc, ignore_index = True)
<ipython-input-71-07efbceb0bb1>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc, ignore_index = True)
<ipython-input-71-07efbceb0bb1>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

error - 영진코퍼레이션


<ipython-input-71-07efbceb0bb1>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(doc, ignore_index = True)


,name,return
7,명신산업,1.102294
0,국보디자인,0.381533
4,LG화학,0.232679
1,싸이버원,-0.145379
5,대림비앤코,-0.420065
2,노바렉스,-0.509602
3,하나금융20호스팩,NaN
6,휴럼,NaN
